In [1]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import jaccard_score

# Load the grayscale fingerprint image
image = cv2.imread('abcd.jpg', cv2.IMREAD_GRAYSCALE)
if image is None:
    raise FileNotFoundError("Error: 'abcd.jpg' not found.")

# Convert to binary
_, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# Define kernel for morphological operations
kernel = np.ones((3, 3), np.uint8)

# Apply morphological operations
operations = {
    "Eroded": cv2.erode(binary, kernel, iterations=1),
    "Dilated": cv2.dilate(binary, kernel, iterations=1),
    "Opened": cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel),
    "Closed": cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel),
}

# Evaluation function
def evaluate(img, gt):
    result_flat = img.flatten() // 255
    gt_flat = gt.flatten() // 255
    ssim_score = ssim(gt, img)
    iou_score = jaccard_score(gt_flat, result_flat)
    return ssim_score, iou_score

# Evaluate all processed results
scores = {}
for name, img in operations.items():
    ssim_score, iou_score = evaluate(img, binary)
    scores[name] = (ssim_score, iou_score)
    print(f"{name} -> SSIM: {ssim_score:.4f}, IoU: {iou_score:.4f}")

# Identify the best match based on SSIM and IoU
best_name, (best_ssim, best_iou) = max(scores.items(), key=lambda x: (x[1][0], x[1][1]))
best_img = operations[best_name]
print(f"\n✅ Most accurate result: {best_name} (SSIM: {best_ssim:.4f}, IoU: {best_iou:.4f})")

# Annotate the best result with an arrow and text
annotated = cv2.cvtColor(best_img, cv2.COLOR_GRAY2BGR)
arrow_start = (20, 20)
arrow_end = (100, 100)
cv2.arrowedLine(annotated, arrow_start, arrow_end, (0, 0, 255), 2, tipLength=0.3)
cv2.putText(annotated, f"Best: {best_name}", (arrow_end[0] + 10, arrow_end[1]),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

# Display all images
cv2.imshow("Original", image)
cv2.imshow("Binary", binary)
for name, img in operations.items():
    cv2.imshow(name, img)
cv2.imshow("Best Annotated", annotated)

cv2.waitKey(0)
cv2.destroyAllWindows()


Eroded -> SSIM: 0.5279, IoU: 0.7886
Dilated -> SSIM: 0.5326, IoU: 0.8250
Opened -> SSIM: 0.8986, IoU: 0.9622
Closed -> SSIM: 0.8736, IoU: 0.9579

✅ Most accurate result: Opened (SSIM: 0.8986, IoU: 0.9622)
